In [52]:
import pandas as pd
# Load member information
#all_accounts_df = pd.read_csv('~/data/ms_exports/accounts-2024-09-26.csv')
all_accounts_df = pd.read_csv('~/data/test/accounts.csv')
# Load visit data

#visits_df = pd.read_csv('~/data/ms_exports/2024-visits-2024-09-20.csv')
visits_df = pd.read_csv('~/data/test/pool_visits.csv')
# Replace spaces with underscores and convert to lower case
all_accounts_df.columns = all_accounts_df.columns.str.replace(' ', '_', regex=False).str.lower()
# Standardize 'acct_type' and 'payment_status' values
all_accounts_df['acct_type'] = all_accounts_df['acct_type'].str.lower().str.strip()
all_accounts_df['payment_status'] = all_accounts_df['payment_status'].str.lower().str.strip()
all_accounts_df.rename(columns={'acct_#': 'acct_id'}, inplace=True)
# Replace spaces with underscores and convert to lower case
visits_df.columns = visits_df.columns.str.replace(' ', '_', regex=False).str.lower()
visits_df.head()


,checkin_time,acct_id,account_type,name,member_type,type,num_credits
0,2024-05-25 10:02:18,10727,August Pass (Member),Evelyn Blackman,Guest Pass,member,4
1,2024-05-25 10:02:34,10620,Waitlist,Elijah Negron,Child,guest,2
2,2024-05-25 10:02:49,10543,Offered Membership,Judy Carter,Adult,member,4
3,2024-05-25 10:02:53,10386,August Pass (Waitlist),William Spencer,Child,guest,4
4,2024-05-25 10:04:05,10534,August Pass (Member),Timothy Klein,Guest Pass,member,5


In [53]:
num_rows = len(all_accounts_df)
print(num_rows, " Total Number of Accounts in Membersplash")
num_rows = len(visits_df)
print(num_rows, " Total Number of Visits in Membersplash")

800  Total Number of Accounts in Membersplash
39770  Total Number of Visits in Membersplash


In [54]:
# Get unique account types
unique_account_types = all_accounts_df['acct_type'].unique()
print("Unique account types:")
print(unique_account_types)
unique_account_types_vists = visits_df['account_type'].unique()
print("Unique account types in Visits:")
print(unique_account_types_vists)

Unique account types:
['gold pass' 'active summer' 'august pass (waitlist)' 'offered membership'
 'honorary pass' 'waitlist' 'august pass (non member)'
 'august pass (member)']
Unique account types in Visits:
['August Pass (Member)' 'Waitlist' 'Offered Membership'
 'August Pass (Waitlist)' 'Honorary Pass' 'August Pass (Non Member)'
 'Active Summer' 'Gold Pass']


In [55]:
# Calculate the counts of each account type
account_type_counts = all_accounts_df['acct_type'].value_counts()
print("Total number of members for each account type:")
print(account_type_counts)

Total number of members for each account type:
acct_type
gold pass                   118
active summer               112
offered membership          105
waitlist                    100
honorary pass                98
august pass (waitlist)       91
august pass (non member)     90
august pass (member)         86
Name: count, dtype: int64


## Account Type Definitions 
### (Large update each Fall and February - whenever people pay)

- **Active Summer**: Eligible for Summer Membership - Includes Paid and Unpaid accounts
- **Gold Pass**: Limited Use Membership
- **Waitlist**: Paid to be on the Waitlist (Includes people who declined or didn't respond for the second time in 2024)
- **Offered Membership**: 
- **Honorary Pass**: 
- **Inactive**: 
- **Test Admin**: 

### August Pass Types

- **August Pass (Waitlist)**: People on the Waitlist who have purchased an August Pass
- **August Pass (Non Member)**: People who purchased an August Pass who are neither on the Waitlist nor a Member
- **August Pass (Member)**: People who are Members who have purchased an August Pass
- **Non-Members (August Pass)**: People who have previously purchased an August Pass

In [56]:
# Define the target account types
august_target_account_types = ['August Pass (Waitlist)', 'August Pass (Member)','August Pass (Non Member)']
# Filter members based on the target account types
filtered_members_df = all_accounts_df[all_accounts_df['acct_type'].isin(august_target_account_types)]
# Identify unique account #s
all_members = set(filtered_members_df['acct_id'].unique())
print(len(all_members), " Total August Pass Accounts")

#TODO: Filter out 43 people Unpaid 


0  Total August Pass Accounts


In [57]:
# Get unique member IDs who visited
visiting_member_ids = visits_df['acct_id'].unique()
# Create a DataFrame indicating whether each member_id has visited
visited_df = pd.DataFrame({'acct_id': visiting_member_ids})
visited_df['visited'] = True

# Merge members_df with visited_df
members_visited_df = all_accounts_df.merge(visited_df, on='acct_id', how='left')

# Fill NaN values in 'visited' column with False (members who did not visit)
members_visited_df['visited'] = members_visited_df['visited'].fillna(False)
print(members_visited_df.head)

<bound method NDFrame.head of      acct_id                 acct_type  \
0      10000                 gold pass   
1      10001             active summer   
2      10002    august pass (waitlist)   
3      10003        offered membership   
4      10004    august pass (waitlist)   
..       ...                       ...   
795    10795    august pass (waitlist)   
796    10796  august pass (non member)   
797    10797                 gold pass   
798    10798  august pass (non member)   
799    10799    august pass (waitlist)   

                                        tags date_registered  legacy_acct_#  \
0                        Extended Membership      2024-03-14            NaN   
1                        Extended Membership      2024-01-22         5218.0   
2                        Extended Membership      2023-11-30            NaN   
3    Gold Pass Eligible, Extended Membership      2023-12-24            NaN   
4                                        NaN      2024-06-26          

In [60]:
# Create a crosstab of 'acct_type' and 'visited'
#visit_crosstab = pd.crosstab(members_visited_df['acct_type'], members_visited_df['visited'])

# Create a crosstab of 'acct_type', 'payment_status', and 'visited'
visit_crosstab = pd.crosstab(
    [members_visited_df['acct_type'], members_visited_df['payment_status']],  # Rows: acct_type and payment_status
    members_visited_df['visited']  # Columns: visited status
)

# Reset index to turn 'acct_type' and 'payment_status' back into columns
visit_crosstab = visit_crosstab.reset_index()

# Rename columns for clarity
visit_crosstab = visit_crosstab.rename(columns={False: 'did_not_visit', True: 'visited'})

# Ensure both 'visited' and 'did_not_visit' columns exist
if 'did_not_visit' not in visit_crosstab.columns:
    visit_crosstab['did_not_visit'] = 0

if 'visited' not in visit_crosstab.columns:
    visit_crosstab['visited'] = 0


In [61]:

# Calculate total members per account type
visit_crosstab['total_accounts'] = visit_crosstab['visited'] + visit_crosstab['did_not_visit']

# Reorder columns for better readability
#visit_crosstab = visit_crosstab[['acct_type', 'total_accounts', 'visited', 'did_not_visit']]
# Reorder columns
visit_crosstab = visit_crosstab[['acct_type', 'payment_status', 'total_accounts', 'visited', 'did_not_visit']]


# Display the result
print(visit_crosstab)


visited                 acct_type payment_status  total_accounts  visited  \
0                   active summer           paid              55       55   
1                   active summer         unpaid              57       57   
2            august pass (member)           paid              39       39   
3            august pass (member)         unpaid              47       47   
4        august pass (non member)           paid              48       48   
5        august pass (non member)         unpaid              42       42   
6          august pass (waitlist)           paid              44       44   
7          august pass (waitlist)         unpaid              47       47   
8                       gold pass           paid              64       64   
9                       gold pass         unpaid              54       54   
10                  honorary pass           paid              49       49   
11                  honorary pass         unpaid              49       49   

In [30]:
# Define the target account type
target_account_type = 'active summer'  # Replace with the actual account type from your data

# Filter members who belong to the target account type and did not visit
unpaid_visiting_members = members_visited_df[
    (members_visited_df['acct_type'] == target_account_type) &
    (members_visited_df['visited'] == True) &
    (members_visited_df['payment_status'] == 'unpaid')
]

# Display the non-visiting member(s)
print(f"\Visiting member(s) in account type '{target_account_type}':")
print(unpaid_visiting_members)

\Visiting member(s) in account type 'active summer':
     acct_id      acct_type                tags date_registered  \
352    10380  active summer                 NaN      2011-05-01   
382    10416  active summer  Gold Pass Eligible      2005-05-01   
555    10599  active summer  Gold Pass Eligible      2000-05-01   

     legacy_acct_#                      user_name first_name  last_name  \
352         3017.0  josh@joshgreenberglawfirm.com       Josh  Greenberg   
382         2563.0          kohls_heavner@msn.com      Brett    Heavner   
555         2260.0          stephen.marino@me.com    Stephen     Marino   

                             email  site_access  ...    zip    cell_phone  \
352  josh@joshgreenberglawfirm.com            1  ...  22207           NaN   
382          kohls_heavner@msn.com            1  ...  22207  703-869-7301   
555          stephen.marino@me.com            1  ...  22205  202-841-2941   

    work_phone    home_phone payment_status checkin_note #_members  

In [31]:
# Define the target account type
target_account_type = 'Waitlist'  # Replace with the actual account type from your data

# Filter members who belong to the target account type and did not visit
waitlist_visiting_members = members_visited_df[
    (members_visited_df['acct_type'] == target_account_type) &
    (members_visited_df['visited'] == True)
]

# Display the non-visiting member(s)
print(f"\Visiting member(s) in account type '{target_account_type}':")
print(waitlist_visiting_members)


\Visiting member(s) in account type 'Waitlist':
Empty DataFrame
Columns: [acct_id, acct_type, tags, date_registered, legacy_acct_#, user_name, first_name, last_name, email, site_access, directory, guest_credits, billing_member_email, street_address, street_address_2, city, state, zip, cell_phone, work_phone, home_phone, payment_status, checkin_note, #_members, authnet_account, member_names, visited]
Index: []

[0 rows x 27 columns]


In [32]:
waitlist_visits = visits_df[
    (visits_df['acct_id'] == 10599)
]
print(waitlist_visits)
# August Pass (Waitlist) moved back

              checkin_time  acct_id   account_type           name member_type  \
19156  2024-07-16 14:16:39    10599  Active Summer  Alison Marino       Adult   

         type  num_credits  
19156  member            0  
